In this practical session, you will implement different strategies to build a recommender system.

# Dataset



We will use The Movie Dataset, available on Kaggle.  
It contains metadata for all 45,000 movies listed in the [Full MovieLens Dataset](https://grouplens.org/datasets/movielens/). The dataset consists of movies released on or before July 2017. Data points include cast, crew, plot keywords, budget, revenue, posters, release dates, languages, production companies, countries, TMDB vote counts and vote averages.

This dataset also has files containing 26 million ratings from 270,000 users for all 45,000 movies. Ratings are on a scale of 1-5 and have been obtained from the official GroupLens website.  
You will need [Kaggle](https://www.kaggle.com/) account to download the data.  You should already have one since the DEFI IA is hosted on Kaggle this year. If you don't, it is time to create your account (and to start participating to the DEFI ;-) )  
Once you are logged into Kaggle, go to your account and scroll down to the API section to generate a new token.  
![](https://drive.google.com/uc?export=view&id=1YcSTHD_FGrwDKaaLk6T9Gsdte8TKuPCt)  
We will now install the kaggle library to download the dataset directly from the notebook.



In [ ]:
!pip install kaggle

Run the next cell to upload your token.

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  
# Then move kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

We will start by working with the metadata dataset.
It contains information about the movies like their title, description, genres, or even their average IMDB ratings.

In [ ]:
!kaggle datasets download "rounakbanik/the-movies-dataset" -f movies_metadata.csv
!kaggle datasets download "rounakbanik/the-movies-dataset" -f ratings.csv
!unzip movies_metadata.csv.zip
!unzip ratings.csv.zip

Use pandas to explore the *movies_metadata.csv* dataset.

In [ ]:
import pandas as pd
metadata = pd.read_csv('movies_metadata.csv')
metadata.dropna(subset=['title'], inplace=True)
metadata['id'] = pd.to_numeric(metadata['id'])
metadata['genres'] = metadata['genres'].apply(lambda x: ' '.join([i['name'] for i in eval(x)]))
metadata.head(5)

Create a new column called _year_ and use seaborn to plot the number of movies per year.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

metadata['year'] = ...

plt.figure(figsize=(20,10))
...

# Recommendation by popularity

The metadata dataset contains informations about ratings in the _vote_average_ column.  
A classical baseline, or cold start when you implement a recommender system consists in using popular products.  
## Best movies by average note  
Try to visualize the movies with the best vote average.
Do you know these movies?

In [ ]:
...

You may have guessed that the average score is only reliable when it is averaged on a sufficient number of votes.  
Use seaborn ```histplot``` method to plot the histogram of the number of votes.
For better readability you may first do this plot for the movies with less than 100 votes and then do another ones for the remaining ones.

In [ ]:
plt.figure(figsize=(20,10))
plt.subplot(2,1,1)
sns.histplot(...)
plt.title('Vote count')
plt.subplot(2,1,2)
sns.histplot(...)
plt.title('Vote count')

Try to visualize the best movies according to the average vote for movies that have at least 1000 votes.
You should now know some of these movies.


In [ ]:
...

## Best movies by IMDb score  
IMDb (an acronym for Internet Movie Database) is an online database of information related to films, television programs, home videos, video games, and streaming content online.  
It might be considered as one of the most exhaustive databases on movies.  
In addition, IMDb maintains a ranking of movies according to people's votes. To do so, it computes a score based on the average rating and the number of votes. 
The formula they are using is described [here](https://help.imdb.com/article/imdb/track-movies-tv/ratings-faq/G67Y87TFYYP6TWAV#)  
![](https://drive.google.com/uc?export=view&id=12J_uJ86eOimr8Y0LHTGSMmUgkBnZu9cO)   
Use this formula to compute the IMDb score for all movies and visualize the ones with the best scores. (You may use a smaller value for m, 500 for example)


In [ ]:
m = 500
c = ...

def imdb_score(x):
    ...
    score = ...
    return score

metadata['imdb_score'] = ...
...

What were the best movies in your birth year?

In [ ]:
birth_year = ...
...

The following code will create a data frame containing one-hot encoding of the movie's genre.  
Use it to recommend the best movies according to the genre and the IMDB score (for example the best Horror movies)

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer

metadata['genres_list'] = metadata['genres'].apply(lambda s: s.split(" "))
mlb = MultiLabelBinarizer()
genre_df = pd.DataFrame(mlb.fit_transform(metadata['genres_list'].fillna('[]')),columns=mlb.classes_, index=metadata.index)
genre_df.head()

In [ ]:
...

# Content based recommender systems

### Item description
Another way to create a recommender system is to base the recommendations on the content.
It is an exciting way to start a recommender system when you do not have many user interactions or new items to recommend.  
In many cases, the text description is a good starting point.
Use what you learned from the first practical section on text data to compute a TF-IDF matrix with the descriptions of the movies (since colab has limited RAM, use a max of 4000 features. We will also work on a subset of the dataset using only the film that were displayed after 2000). 

In [ ]:
metadata['overview'] = metadata['overview'].fillna('')
subset = metadata[metadata['release_date'] > "2000"].reset_index()
subset['overview'].head()

In [ ]:
from nltk import word_tokenize          
from nltk.stem import WordNetLemmatizer
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

...
...
...

# Create TF-idf model
tfidf = ...

#Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix = ...



Now that you have a representation computed for each movie, you can calculate distances or similarities for movie pairs.
Compute the cosine similarity matrix of your TF-IDF Matrix.  
You may use scikit-learn 's [cosine_distances](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.cosine_distances.html) function.

In [ ]:
from sklearn.metrics.pairwise import cosine_distances
cosine_sim = ...

We will create a list containig the movies with the correct indexes to help us recommended movies.

In [ ]:
titles = subset['title']
indices = pd.Series(subset.index, index=subset['title'])
titles[370:390]

Use the following function with your similarity matrix to recommend movies from another movie title.

In [ ]:
def get_reco(title, sim_matrix):
  idx = indices[title]
  print(f'original: {title}')
  recos = sim_matrix[idx].argsort()[1:6]
  recos = titles.iloc[recos]

  print(recos)

title = 'The Dark Knight Rises'#'Rush Hour 2'
get_reco(...)

Delete the similarity matrix to free some meomry in the Colab instance.



In [ ]:
del(cosine_sim)

### Item attributes
Sometimes your catalog is also filled with additional information about the items.  
These pieces of information are usually hand filled and may contain insightful features for a content-based recommender system.  
In our case we will download an associated dataset containing informations about the movie casting and the production crew and an other dataset containing keywords associated to the movies.

In [ ]:
!kaggle datasets download "rounakbanik/the-movies-dataset" -f credits.csv
!unzip credits.csv.zip

In [ ]:
credits = pd.read_csv('credits.csv')
credits['cast'] = credits['cast'].apply(lambda x: ' '.join([i['name'].replace(' ', '') for i in eval(x)]))
credits['crew'] = credits['crew'].apply(lambda x: ' '.join([i['name'].replace(' ', '') for i in eval(x)]))
credits.head()

In [ ]:
!kaggle datasets download "rounakbanik/the-movies-dataset" -f keywords.csv
!unzip keywords.csv.zip

In [ ]:
keywords = pd.read_csv('keywords.csv')
keywords['keywords'] = keywords['keywords'].apply(lambda x: ' '.join([i['name'] for i in eval(x)]))
keywords.head()

We will now create another dataframe containing all the movies attributes.

In [ ]:
attributes_df = pd.merge(keywords, credits, on='id')
attributes_df = pd.merge(attributes_df, metadata, on='id')
attributes_df = attributes_df.sort_values('vote_count', ascending=False).drop_duplicates(subset='id').reset_index()
# We will aslo use a subset to avoid Out of Memory issues
attributes_df = attributes_df[attributes_df['release_date'] > "2000"].reset_index()
attributes_df[['title', 'genres', 'cast', 'crew', 'keywords']].head()

Create a new columns called *attributes* where you will concatenate the genre, the cast, the crew and the keywords.

In [ ]:
attributes_df['attributes'] = ...

Now repeat the previous feature extraction by TF-IDF on this column and compute a new similarity matrix.

In [ ]:
tfidf = TfidfVectorizer(...)
tfidf_matrix = ...
cosine_sim = ...

We may need to re-create our tiltle index dataframe.

In [ ]:
titles = attributes_df['title']
indices = pd.Series(attributes_df.index, index=attributes_df['title'])
titles[370:390]

Try your new matrix similarity to recommend movies based on these new attributes.

In [ ]:
title = 'Rush Hour 2'#'Inception'
...

Let's free some meomry in the Colab instance.

In [ ]:
del(cosine_distances)
del(tfidf_matrix)
del(attributes_df)

### Images

An other type of content may be one or several images of the products. 
It may not necessarily be relevant in the case of movies but let's do it anyway.  
We will now work with images and recommend movies according to their posters.  
We first need to download another dataset.

In [ ]:
!kaggle datasets download "ghrzarea/movielens-20m-posters-for-machine-learning"
!unzip movielens-20m-posters-for-machine-learning.zip

The following code will allow us to load the data.

In [ ]:
# taken from  andrewjong/pytorch_image_folder_with_file_paths.py (https://gist.github.com/andrewjong/6b02ff237533b3b2c554701fb53d5c4d) 

import torch
from torchvision import datasets

class ImageFolderWithPaths(datasets.ImageFolder):
    """Custom dataset that includes image file paths. Extends
    torchvision.datasets.ImageFolder
    """

    # override the __getitem__ method. this is the method that dataloader calls
    def __getitem__(self, index):
        # this is what ImageFolder normally returns 
        original_tuple = super(ImageFolderWithPaths, self).__getitem__(index)
        # the image file path
        path = self.imgs[index][0]
        # make a new tuple that includes original and the path
        tuple_with_path = (original_tuple + (path,))
        return tuple_with_path



We will use a pre-trained network to extract the features from the posters.   
Similar to what we did with the text descriptions, we will compute similarities between the movies according to these features.  

The pre-trained model we will be using was trained with normalized images. Thus, we have to normalize our posters before feeding them to the network.  
The following code will instantiate a data loader with normalized images and provide a function to revert the normalization for visualization purposes.


In [ ]:
from torch.utils.data import DataLoader
import torchvision.transforms as transforms

mean = [ 0.485, 0.456, 0.406 ]
std = [ 0.229, 0.224, 0.225 ]
normalize = transforms.Normalize(mean, std)
inv_normalize = transforms.Normalize(
   mean= [-m/s for m, s in zip(mean, std)],
   std= [1/s for s in std]
)

transform = transforms.Compose([transforms.Resize((224, 224)),
                                transforms.ToTensor(),
                                normalize])
dataset = ImageFolderWithPaths('MLP-20M', transform)    
    
dataloader = DataLoader(dataset, batch_size=128, num_workers=2)

Here are some exemples of posters:

In [ ]:
from torchvision.utils import make_grid
import matplotlib.pyplot as plt
x, _, paths = next(iter(dataloader))
img_grid = make_grid(x[:16])
img_grid = inv_normalize(img_grid)
plt.figure(figsize=(20,15))
plt.imshow(img_grid.permute(1, 2, 0))
plt.axis('off')

Instantiate a pre-trained a mobilenet_v3_small model (documentation [here](https://pytorch.org/vision/stable/models.html))

In [ ]:
import ...
mobilenet = ...

Have a look to the model:

In [ ]:
print(mobilenet)

We will now crate a subset of this model to extract the features.  
Use a Sequential model to get only the features followed by the avgpool layer of mobilnet and finish with a Flatten layer (```torch.nn.Flatten()```)


In [ ]:
model = torch.nn.Sequential(...).cuda()

If your model is OK, it should predict 576-dimensional vectors.

In [ ]:
import torch
x = torch.zeros(100, 3, 224,224).cuda()
y = model(x)
y.shape

We will now create a dataframe with our extracted features and the path to the poster image.

In [ ]:
import pandas as pd
from tqdm.notebook import tqdm

df = pd.DataFrame(columns=["features", "path"])
for x, _, paths in tqdm(dataloader):
  with torch.no_grad():
    x = x.cuda()
    features = ...
  tmp = pd.DataFrame({'features': list(features.cpu().numpy()), 'path': list(paths)})
  df = df.append(tmp, ignore_index=True)
df.head()

We will now extract all the features into a numpy array that will be used to compute the similarity matrix.

In [ ]:
import numpy as np
features = np.vstack(df.features)
features.shape

Now compute the cosine similarity between your features.

In [ ]:
from sklearn.metrics.pairwise import cosine_distances
cosine_sim = ...
cosine_sim.shape

The ```plot_image``` function  will display a poster according to it's path.  
Fill the ```plot_images``` function to plot a series of posters from a list of paths

In [ ]:
import matplotlib.image as mpimg

def plot_image(path):
  img = mpimg.imread(path)
  plt.imshow(img)
  plt.axis('off')

def plot_images(paths_list):
  plt.figure(figsize=(20,20))
  n = len(paths_list)
  for i, path in enumerate(paths_list):
    ...


plot_images(['MLP-20M/MLP-20M/1.jpg', 'MLP-20M/MLP-20M/2.jpg', 'MLP-20M/MLP-20M/3.jpg', 'MLP-20M/MLP-20M/4.jpg', 'MLP-20M/MLP-20M/5.jpg'])

Fill the following code to implement a function that will plot the top 5 recommendations for a movie according to its index.

In [ ]:
def plot_reco(idx, sim_matrix):
  img = plot_image(df['path'][idx])
  recos = sim_matrix[idx].argsort()[1:6]
  reco_posters = df.iloc[recos]['path'].tolist()
  plot_images(reco_posters)

idx = 16 #10 #200
plot_reco(...)

Try with different movie indexes, you will be surprised by the lack of originality of the marketing staffs ;-)

# Collaborative filtering

### Item-Item

Item-item collaborative filtering, is a form of collaborative filtering for recommender systems based on the similarity between items calculated using people's ratings. 
For sake of simplicity, in this practical session, we will only focus on item-item similarity methods.
If you have time, feel free to try an user-item approach. The following [blog post](https://notebook.community/saksham/recommender-systems/Collaborative%20Filtering) may help you to do it.

We will use another dataset containing the ratings of several users on movies.

In [ ]:
!wget https://raw.githubusercontent.com/wikistat/AI-Frameworks/master/RecomendationSystem/movielens_small/movies.csv
!wget https://raw.githubusercontent.com/wikistat/AI-Frameworks/master/RecomendationSystem/movielens_small/ratings.csv

In [ ]:
ratings = pd.read_csv('ratings.csv')
ratings = ratings.rename(columns={'movieId':'id'})
ratings['id'] = pd.to_numeric(ratings['id'])
ratings = pd.merge(ratings, metadata[['title', 'id']], on='id')[['userId', 'id', 'rating', 'title']]
ratings.head()

In [ ]:
ratings.shape

This dataset is a bit huge and may slow down futur computations. Moreover collaborative filtering kind of suffers from products or user with few ratings.  
We will only focus on the 100 movies with the most ratings and the users with the highest number of ratings.

In [ ]:
# filter movies
ratings['count'] = ratings.groupby('id').transform('count')['userId']
movieId = ratings.drop_duplicates('id').sort_values(
    'count', ascending=False).iloc[:100]['id']
ratings = ratings[ratings['id'].isin(movieId)].reset_index(drop=True)

#filter users
ratings['count'] = ratings.groupby('userId').transform('count')['id']
userId = ratings.drop_duplicates('userId').sort_values(
    'count', ascending=False).iloc[:20001]['userId']
ratings = ratings[ratings['userId'].isin(userId)].reset_index(drop=True)

ratings.shape

In [ ]:
ratings.head()

In [ ]:
ratings.title.unique()

Now, we need to build a pivot table with user in lines, movies in columns and ratings as values.  
Use pandas [pivot_table](https://pandas.pydata.org/docs/reference/api/pandas.pivot_table.html) method to create this pivot table.

In [ ]:
pivot = ...
pivot.head(100)

With this pivot table, it is now easy to compute the similarity between movies.  
Indeed each movie can be represented by a vector of the users' ratings.
Instead of using a cosine similarity distance as we did earlier in the notebook, we will use the Pearson correlation score since it is already implemented in Pandas.  
The pivot table has a method ```corrwith``` that will return the Pairwise correlation score of one entry with all entries of the table.

In [ ]:
movie_vector = pivot["The Bourne Supremacy"]
#movie_watched = pivot["Solo: A Star Wars Story (2018)"]
similarity = ...
similarity.head()

Sort the produced results to get the best recommendations to The Bourne Supremacy. 
You may also try with different movies.

In [ ]:
...

## Matrix factorization
Matrix factorization is certainly one of the most efficient way to build a recomender system. I really encourage you to have a look to [this article](https://datajobs.com/data-science-repo/Recommender-Systems-%5BNetflix%5D.pdf) presenting the matrix factorization techniques used in recommender systems.

The idea is pretty simple, it consists in factorizing the ratings matrix $R$ into the product of a user embedding matrix $U$ and an item embedding matrix $V$, such that $R \approx UV^\top$ with
$U = \begin{bmatrix} u_{1} \\ \hline \vdots \\ \hline u_{N} \end{bmatrix}$ and
$V = \begin{bmatrix} v_{1} \\ \hline \vdots \\ \hline v_{M} \end{bmatrix}$.

Where
- $N$ is the number of users,
- $M$ is the number of items,
- $R_{ij}$ is the rating of the $j$th item by the $i$th user,
- each row $U_i$ is a $d$-dimensional vector (embedding) representing user $i$,
- each row $V_j$ is a $d$-dimensional vector (embedding) representing item $j$,


One these emmbeding matrices are built, predicting the rating of an user $i$ for an item $j$ consists in computing the dot product $\langle U_i, V_j \rangle$.

### Using surpise

![](https://drive.google.com/uc?export=view&id=1dh2RJ95F0j-rZyuf59G35239B42veAWD) 

We will begin by using the famous Singular Value Decomposition method.
Several libraries implement this algorithm.
In this session, we will be using [Surprise](http://surpriselib.com/).
Surprise is a recommender system library implemented in Python.  
It was actually developed by [Nicolas Hug](http://nicolas-hug.com/about) an INSA Toulouse Alumni!

In [ ]:
!pip install scikit-surprise

Surprise implements the SVD algorithm.  Help yourself with [the doc](https://surprise.readthedocs.io/en/stable/getting_started.html) to train an SVD model on the rating dataset.  

In [ ]:
#Creating a train and a test set
testset = ratings.sample(frac=0.1, replace=False)
trainset = ratings[~ratings.index.isin(testset.index)]

assert set(testset.userId.unique()).issubset(trainset.userId.unique())
assert set(testset.id.unique()).issubset(trainset.id.unique())

In [ ]:
from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate

In [ ]:
reader = Reader(rating_scale=(0, 5))
data = Dataset.load_from_df(ratings[['userId', 'id', 'rating']].fillna(0), reader)
svd = SVD()
...

Let us look some ratings for one user in the test dataset.

In [ ]:
testset[testset['userId'] == 24]

What would your model predict for these exemples?

In [ ]:
uid = 24
iid = 3114

...

Write a code to recommend 5 movies to an user.

In [ ]:
...

### Using gradient descent
Another way to compute the matrix factorization consists in using gradient descent to minimize $\text{MSE}(R, UV^\top)$ where:

$$
\begin{align*}
\text{MSE}(A, UV^\top)
&= \frac{1}{|\Omega|}\sum_{(i, j) \in\Omega}{( R_{ij} - (UV^\top)_{ij})^2} \\
&= \frac{1}{|\Omega|}\sum_{(i, j) \in\Omega}{( R_{ij} - \langle U_i, V_j\rangle)^2}
\end{align*}
$$
where $\Omega$ is the set of observed ratings, and $|\Omega|$ is the cardinality of $\Omega$.

We will now implement our own matrix factorization algorith using Pytorch.
To do so we first need to convert our ratings datasets in Pytorch datasets.

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader

user_list = trainset.userId.unique()
item_list = trainset.id.unique()
user2id = {w: i for i, w in enumerate(user_list)}
item2id = {w: i for i, w in enumerate(item_list)}

class Ratings_Datset(Dataset):
    def __init__(self, df):
        self.df = df.reset_index()

    def __len__(self):
        return len(self.df)
  
    def __getitem__(self, idx):
        user = user2id[self.df['userId'][idx]]
        user = torch.tensor(user, dtype=torch.long)
        item = item2id[self.df['id'][idx]]
        item = torch.tensor(item, dtype=torch.long)
        rating = torch.tensor(self.df['rating'][idx], dtype=torch.float)
        return user, item, rating


trainloader = DataLoader(Ratings_Datset(trainset), batch_size=512, shuffle=True ,num_workers=2)
testloader = DataLoader(Ratings_Datset(testset), batch_size=64, num_workers=2)

These dataloader will provide mini-batches of tuples <user, movie, rating>.
We will use a special type of Pytorch layers call [Embedding](https://pytorch.org/docs/stable/generated/torch.nn.Embedding.html).  
These layers will create a mapping between an index and a vector representation.  
In our case they will provide vector representations of our users and items.  
We will train the matrix factorization model to minimize the prediction error between a rating and the dot product of an user embedding with a movie embedding.  
![](https://drive.google.com/uc?export=view&id=1wSQbcSN_I28mF74-wnb8_qjAzRH9YDjA) 

Complete the following code to implement the ```MatrixFactorization``` class in Pytorch.

In [ ]:
import torch

class MatrixFactorization(torch.nn.Module):
    def __init__(self, n_users, n_items, n_factors=20):
        super().__init__()
        self.user_embeddings = torch.nn.Embedding(...)
        self.item_embeddings = torch.nn.Embedding(...)

    def forward(self, user, item):
        user_emb = ...
        item_emb = ...
        return torch.mul(user_emb, user_emb).sum(1)


Complete the training method that we will use to train the network.

In [ ]:
from tqdm.notebook import tqdm
import torch
import torch.nn as nn
from statistics import mean


def train(model, optimizer, trainloader, epochs=30):
    criterion = ...
    t = tqdm(range(epochs))
    for epoch in t:
        corrects = 0
        total = 0
        train_loss = []
        for users, items, r in trainloader:
            users = users.cuda()
            items = items.cuda()
            r = r.cuda() / 5 #We normalize the score to ease training
            y_hat = ...
            loss = criterion(y_hat, r.unsqueeze(1).float())
            train_loss.append(loss.item())
            total += r.size(0)
            ...
            ...
            ...
            t.set_description(f"loss: {mean(train_loss)}")

We now have everything to train our model.
Train your model with an Adam optimizer (lr=1e-3) for 5 epochs.

In [ ]:
n_user = trainset.userId.nunique()
n_items = trainset.id.nunique()
model = ...
optimizer = ...
train(...)

Complete the following code to evaluate your model.

In [ ]:
import math

def test(model, testloader, m_eval=False):

    
    running_mae = 0
    with torch.no_grad():
        corrects = 0
        total = 0
        for users, items, y in testloader:
            users = ...
            items = items.cuda()
            y = y.cuda() / 5
            y_hat = ...
            error = torch.abs(y_hat - y).sum().data
            
            running_mae += ...
            total += y.size(0)
    
    mae = ...
    return mae * 5
    

test(model, testloader)

Try to compare the predictions of your model with actual ratings.

In [ ]:
users, movies, r = next(iter(testloader))
users = users.cuda()
movies = movies.cuda()
r = r.cuda()

pred = ...
print("ratings", r[:10].data)
print("predictions:", pred.flatten()[:10].data)

We just trained a matrix factorization algorithm using Pytorch.  
In this setting, the final prediction was made with the dot product of our embeddings.
Actually with a minimal modification of the Class, we could create a full neural network.  
If we replace the dot product with a fully-connected network, we would actually have an end-to-end neural network able to predict the ratings of our users.  
![](https://drive.google.com/uc?export=view&id=1THBMB-Z3db0Rn0dyYYWhN98AHcYEM-nT)  
This approach is called Neural Collaborative Filtering and is presented in this [paper](https://arxiv.org/pdf/1708.05031.pdf).  
Try to fill in the following code to create an NCF network.



In [ ]:
class NCF(nn.Module):
        
    def __init__(self, n_users, n_items, n_factors=8):
        super().__init__()
        self.user_embeddings = torch.nn.Embedding(n_users, n_factors)
        self.item_embeddings = torch.nn.Embedding(n_items, n_factors)
        self.predictor = torch.nn.Sequential(
            nn.Linear(in_features=..., out_features=64),
            ...,
            nn.Linear(in_features=32, out_features=1),
            nn.Sigmoid()
        )
        
        
    def forward(self, user, item):
        

        user_emb = ...
        item_emb = ...

        # Concat the two embedding layers
        z = torch.cat([user_emb, item_emb], dim=-1)
        y = 
        return y

Train your NCF network on the train dataset and test it on the test dataset.

In [ ]:
model = NCF(n_user, n_items).cuda()
optimizer = ...
train(model, optimizer, trainloader, epochs=5)

In [ ]:
test(model, testloader)

In [ ]:
users, movies, r = next(iter(testloader))
users = users.cuda()
movies = movies.cuda()
r = r.cuda()

...
print("ratings", r[:10].data)
print("predictions:", ...)

### Implicit feedback with pytorch

In this practical session, we only worked with explicit feedbacks (ratings).
Sometimes you do not have access to such quantitative feedback and have to deal with implicit feedback.  
An implicit feedback is a user's qualitative interaction with an item, such as clicking on an item (positive feedback) or stopping watching a video (negative feedback).
If you are interested in neural collaborative filtering in the case of implicit feedback, I recommend you look at this [excellent tutorial](https://sparsh-ai.github.io/rec-tutorials/matrixfactorization%20movielens%20pytorch%20scratch/2021/04/21/rec-algo-ncf-pytorch-pyy0715.html).